# 股票数据每日增量更新

**用途**：每天运行一次，将 Repository 中所有已有股票更新到最新交易日。

- `END_DATE`：今天
- `START_DATE`：今天往前推 1 个月（足以覆盖所有新增交易日，同时大幅减少下载量）
- 流程：`Provider → DataValidator → compare_and_merge → DataRepository（纯覆盖）`

> 💡 首次下载历史数据请使用 `data_download.ipynb`。
> 本 notebook 只更新 Repository 中**已有**的股票。

## 1. 环境初始化

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from data_process.data_provider  import HybridDataProvider
from data_process.repository     import DataRepository
from data_process.data_updater   import DataUpdater, UpdateResult

provider = HybridDataProvider()
repo     = DataRepository('./rawdata/repository')
updater  = DataUpdater(repo, provider)

print('✅ 初始化完成')

C:\Users\mengxinchen\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


✅ 初始化完成


## 2. 日期参数（自动计算，无需修改）

In [2]:
# END_DATE = 今天；START_DATE = 今天往前推 1 个月
END_DATE   = datetime.today().strftime('%Y-%m-%d')
START_DATE = (datetime.today() - timedelta(days=31)).strftime('%Y-%m-%d')

# ========== 可选覆盖（一般无需修改） ==========
# UPDATE_SYMBOLS = ['600519', '000001']   # None = 更新全部已有股票
# UPDATE_DTYPES  = ['ohlcv', 'adjustment']  # None = 全部类型
UPDATE_SYMBOLS = None
UPDATE_DTYPES  = None
SKIP_ON_ERROR  = True   # 单只失败时继续，不中断整批
# ========== 配置结束 ==========

symbols_in_repo = repo.list_symbols('ohlcv')
target_symbols  = UPDATE_SYMBOLS if UPDATE_SYMBOLS else symbols_in_repo

print(f'📅 更新区间: {START_DATE}  ~  {END_DATE}')
print(f'📦 Repository 中共有 {len(symbols_in_repo)} 只股票')
print(f'🎯 本次更新: {len(target_symbols)} 只股票')
print(f'📋 数据类型: {UPDATE_DTYPES if UPDATE_DTYPES else "全部 (ohlcv / adjustment / market_value / financial)"}')

📅 更新区间: 2026-01-22  ~  2026-02-22
📦 Repository 中共有 3 只股票
🎯 本次更新: 3 只股票
📋 数据类型: 全部 (ohlcv / adjustment / market_value / financial)


## 3. 执行增量更新

In [3]:
print('=' * 80)
print(f'日常增量更新  {START_DATE} ~ {END_DATE}')
print('=' * 80)

update_results = updater.update_all(
    start_date    = START_DATE,
    end_date      = END_DATE,
    data_types    = UPDATE_DTYPES,
    symbols       = UPDATE_SYMBOLS,
    skip_on_error = SKIP_ON_ERROR,
)

print('\n✅ 更新完成')

日常增量更新  2026-01-22 ~ 2026-02-22

✅ 更新完成


## 4. 结果汇总

In [6]:
total          = len(update_results)
success        = sum(1 for r in update_results.values() if r.success)
failed         = total - success
no_new_data    = sum(1 for r in update_results.values() if r.success and r.new_rows == 0 and r.overlap_rows == 0)
warnings_count = sum(1 for r in update_results.values() if r.success and r.warnings)
new_rows_total = sum(r.new_rows for r in update_results.values() if r.success)

print('📊 更新结果统计')
print('=' * 80)
print(f'  总任务数 (股票 × 数据类型): {total}')
print(f'  ✅ 成功:      {success}  （含 {no_new_data} 个已是最新、无新数据）')
print(f'  ⚠️  有警告:   {warnings_count}')
print(f'  ❌ 失败:      {failed}')
print(f'\n  📈 共新增数据行: {new_rows_total}')

# 有警告或失败的明细
flagged = {k: r for k, r in update_results.items() if not r.success or r.warnings}
if flagged:
    print(f'\n⚠️  需关注的任务 ({len(flagged)} 个):')
    print('-' * 80)
    for key, result in flagged.items():
        print(result)
else:
    print('\n✅ 所有任务均正常完成，无警告。')

📊 更新结果统计
  总任务数 (股票 × 数据类型): 12
  ✅ 成功:      12  （含 0 个已是最新、无新数据）
  ⚠️  有警告:   0
  ❌ 失败:      0

  📈 共新增数据行: 0

✅ 所有任务均正常完成，无警告。


## 5. 失败任务明细（如有）

In [7]:
failed_items = {k: r for k, r in update_results.items() if not r.success}

if failed_items:
    print(f'❌ 失败任务 ({len(failed_items)} 个):')
    print('=' * 80)
    for key, r in failed_items.items():
        print(f'  {key}: {r.error}')
    print('\n💡 建议: 重新运行 cell-6，或检查网络 / 数据源是否可用')
else:
    print('✅ 所有任务成功！')

✅ 所有任务成功！
